In [60]:
import scipy.io as sio
import pandas as pd

In [71]:
mat = sio.loadmat('/Users/andrei/Downloads/DREAMER/DREAMER.mat')

In [72]:
mat['DREAMER'].dtype

dtype([('Data', 'O'), ('EEG_SamplingRate', 'O'), ('ECG_SamplingRate', 'O'), ('EEG_Electrodes', 'O'), ('noOfSubjects', 'O'), ('noOfVideoSequences', 'O'), ('Disclaimer', 'O'), ('Provider', 'O'), ('Version', 'O'), ('Acknowledgement', 'O')])

In [73]:
data = mat['DREAMER']['Data'][0, 0][0, 0]
data

array([[(array(['22'], dtype='<U2'), array(['male'], dtype='<U4'), array([[(array([[array([[4390.25641026, 4097.43589744, 4267.69230769, ..., 4512.30769231,
                                4368.71794872, 4287.69230769],
                               [4397.43589744, 4095.8974359 , 4248.20512821, ..., 4506.15384615,
                                4272.82051282, 4187.69230769],
                               [4391.79487179, 4091.28205128, 4261.02564103, ..., 4494.87179487,
                                4342.56410256, 4256.41025641],
                               ...,
                               [4392.82051282, 4107.17948718, 4288.71794872, ..., 4532.82051282,
                                4395.38461538, 4307.17948718],
                               [4385.64102564, 4108.71794872, 4328.20512821, ..., 4542.56410256,
                                4468.71794872, 4431.79487179],
                               [4382.56410256, 4108.71794872, 4385.12820513, ..., 4573.33333333,
       

In [87]:
data['EEG'][0, 0]['baseline'][0, 0][0, 0]

array([[4390.25641026, 4097.43589744, 4267.69230769, ..., 4512.30769231,
        4368.71794872, 4287.69230769],
       [4397.43589744, 4095.8974359 , 4248.20512821, ..., 4506.15384615,
        4272.82051282, 4187.69230769],
       [4391.79487179, 4091.28205128, 4261.02564103, ..., 4494.87179487,
        4342.56410256, 4256.41025641],
       ...,
       [4392.82051282, 4107.17948718, 4288.71794872, ..., 4532.82051282,
        4395.38461538, 4307.17948718],
       [4385.64102564, 4108.71794872, 4328.20512821, ..., 4542.56410256,
        4468.71794872, 4431.79487179],
       [4382.56410256, 4108.71794872, 4385.12820513, ..., 4573.33333333,
        4560.51282051, 4549.23076923]])

In [2]:
from torcheeg.datasets import DREAMERDataset
from torcheeg import transforms
from torcheeg.datasets.constants import DREAMER_CHANNEL_LIST, DREAMER_CHANNEL_LOCATION_DICT
from torcheeg.transforms import after_hook_normalize

In [4]:
dataset = DREAMERDataset(
    io_path="./temp/dreamer",
    mat_path="/Users/andrei/Downloads/DREAMER/DREAMER.mat",
    offline_transform=transforms.Compose(
        [
            # transforms.PickElectrode(
            #     transforms.PickElectrode.to_index_list(
            #         ["F7", "F8", "T7", "T8"], DREAMER_CHANNEL_LIST
            #     ),
            #     apply_to_baseline=True,
            # ),
            transforms.BandDifferentialEntropy(sampling_rate=128,
                                            apply_to_baseline=True),
            transforms.ToGrid(DREAMER_CHANNEL_LOCATION_DICT, apply_to_baseline=True)
        ]
    ),
    online_transform=transforms.Compose([
        transforms.BaselineRemoval(),
        transforms.ToTensor(),
    ]),
    label_transform=transforms.Compose(
        [transforms.Select("valence"), transforms.Binary(3.0)]
    ),
    chunk_size=128,
    baseline_chunk_size=128,
    num_baseline=61,
    num_worker=4,
)

[2025-08-15 20:01:05] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./temp/dreamer.
[2025-08-15 20:01:05] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/23 [00:00<?, ?it/s]
[RECORD 0]: 0it [00:00, ?it/s]
[RECORD 0]: 1it [00:06,  6.19s/it]
[RECORD 0]: 6it [00:06,  1.28it/s]
[RECORD 0]: 10it [00:06,  2.49it/s]
[RECORD 0]: 14it [00:06,  4.04it/s]
[RECORD 0]: 18it [00:06,  6.04it/s]
[RECORD 0]: 23it [00:06,  9.22it/s]
[RECORD 0]: 29it [00:06, 13.81it/s]
[RECORD 0]: 35it [00:06, 19.25it/s]
[RECORD 0]: 41it [00:07, 24.73it/s]
[RECORD 0]: 46it [00:07, 28.84it/s]
[RECORD 0]: 52it [00:07, 34.24it/s]
[RECORD 0]: 58it [00:07, 38.44it/s]
[RECORD 0]: 64it [00:07, 37.90it/s]
[RECORD 0]: 70it [00:07, 42.24it/s]
[RECORD 0]: 76it [00:07, 46.19it/s]
[RECORD 0]: 82it [00:07, 47.22it/s]
[RECORD 0]: 88it

In [5]:
from torcheeg.model_selection import KFoldGroupbyTrial

k_fold = KFoldGroupbyTrial(n_splits=10, random_state=42, shuffle=True,
                           split_path='./temp/split')

In [ ]:
from torch.utils.data import DataLoader
from torcheeg.models import EEGNet, CCNN
from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    model = CCNN(num_classes=2, in_channels=4, grid_size=(9, 9))

    trainer = ClassifierTrainer(
        model=model, num_classes=2, lr=1e-4, weight_decay=1e-4, accelerator="gpu"
    )
    trainer.fit(
        train_loader,
        val_loader,
        max_epochs=50,
        default_root_dir=f"./temp/model/{i}",
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0,
    )
    score = trainer.test(
        val_loader, enable_progress_bar=True, enable_model_summary=True
    )[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')

[2025-08-15 20:06:43] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./temp/split.
[2025-08-15 20:06:43] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | CCNN             | 6.2 M  | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | train
6 | val_metrics   | MetricCollection | 0      | train
7 | test_metrics  | MetricCollection | 0      | train
-----------------------------

Epoch 0:   0%|          | 0/1069 [00:00<?, ?it/s] 

/Users/andrei/miniforge3/envs/tf/lib/python3.11/site-packages/torch/nn/functional.py:1535: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 0: 100%|██████████| 1069/1069 [00:47<00:00, 22.68it/s, v_num=27, train_loss=0.699, train_accuracy=0.540]

[2025-08-15 20:07:36] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.670 train_accuracy: 0.609 


...

[2025-08-15 21:34:18] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.635 train_accuracy: 0.631 


Epoch 14:  91%|█████████ | 979/1073 [00:41<00:03, 23.61it/s, v_num=0, train_loss=0.611, train_accuracy=0.641] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/Users/andrei/miniforge3/envs/tf/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
